In [1]:
import pandas as pd
import geopandas as gpd

/var/folders/2f/fhks6w_d0k556plcv3rfmshw0000gn/T/ipykernel_94787/1259499560.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopanda

In [2]:
chicago_flu = pd.read_csv("tracts.csv").drop(columns=["week", "pop", "illit.r", "den.r"])
deaths_total = chicago_flu.groupby("gisjoin").counts.sum()
chicago_flu = chicago_flu[~chicago_flu.duplicated("gisjoin")].set_index("gisjoin").drop(columns="counts")
chicago_flu["influenza"] = deaths_total
chicago_flu = chicago_flu.rename(columns={c:c.replace(".", "_").lower() for c in chicago_flu.columns})
chicago_flu.index.name = "geography_code"
chicago_flu.index = "G" + chicago_flu.index.astype('str')

In [36]:
chicago_flu.to_csv("../../data_wrangling/data/chicago_influenza_1918.csv")

In [3]:
chicago_flu

,gross_acres,illit,unemployed_pct,ho_pct,agecat1,agecat2,agecat3,agecat4,agecat5,agecat6,agecat7,influenza
geography_code,,,,,,,,,,,,
G17003100001,1388.2,116,0.376950,0.124823,46,274,257,311,222,1122,587,9
G17003100002,217.7,14,0.399571,0.071647,35,320,441,624,276,1061,508,6
G17003100003,401.3,69,0.349558,0.092920,50,265,179,187,163,1020,392,8
G17003100004,86.9,11,0.422535,0.030072,43,241,129,141,123,1407,539,2
G17003100005,337.1,20,0.431822,0.084703,65,464,369,464,328,2625,1213,7
...,...,...,...,...,...,...,...,...,...,...,...,...
G17003100492,2176.6,136,0.404430,0.173351,85,606,520,705,439,2141,1460,12
G17003100493,680.0,271,0.377207,0.130158,243,1349,957,1264,957,4653,2180,40
G17003100494,1392.8,1504,0.336032,0.072317,309,1779,1252,1598,1086,6235,2673,85


In [16]:
gdf = gpd.read_file("shapefile.zip")

In [17]:
gdf["GISJOIN"] = "G" + gdf["GISJOIN"].astype('str')

In [18]:
gdf = gdf[["GISJOIN", "geometry"]].merge(chicago_flu, left_on="GISJOIN", right_index=True)

In [22]:
gdf.rename(columns={"GISJOIN": "geography_code"}).to_file("../../clustering/data/chicago_influenza_1918.geojson")